In [1]:
import os
from amplpy import AMPL, add_to_path

# Add AMPL to path
add_to_path(r"D:/Downloads_Aayush/AMPL software/AMPL")

# Create a custom temporary directory
temp_dir = 'C:/Users/aayush/Desktop/Polyhouse/AMPL_Temp'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Initialize AMPL
ampl = AMPL()

# Set the custom temporary directory for AMPL
ampl.setOption('TMPDIR', temp_dir)

# Set the path to the solver, e.g., Gurobi
ampl.setOption('solver', 'gurobi')


ampl.read('C:/Users/aayush/Desktop/Polyhouse/model.mod')
ampl.readData('C:/Users/aayush/Desktop/Polyhouse/Pol32.dat')

log_file_path = 'C:/Users/aayush/Desktop/Polyhouse/32solver_log.txt'
ampl.setOption('gurobi_options', f'LogFile={log_file_path}')

ampl.solve()

In [2]:
# Create an empty dictionary to store the sets of unique j values
j_dict = {}

# Loop over the i, k, and j sets
for (i, k) in ampl.getSet('IK'):
    for j in ampl.getSet('J'):
        for p in ampl.getSet('P'):
            # Access the variable and check if its value is greater than zero
            x_ikjp = ampl.getVariable('x_ikjp')[i, k, j, p].value()  # Call .value() to get the actual value
            if x_ikjp > 1:  # Now comparing the actual value
                # Add j to the set for the corresponding i and k
                key = (i, k)
                if key not in j_dict:
                    j_dict[key] = set()
                # Add j to the set
                j_dict[key].add(j)

    # Convert the set of unique j values to a list and store it in the dictionary, if key exists
    key = (i, k)
    if key in j_dict:
        j_list = list(j_dict[key])
        # If i is not in the list, move it to index 0
        if i not in j_list:
            j_list.insert(0, i)
        j_dict[key] = j_list

# Output the j_dict for verification
for key, value in j_dict.items():
    print(f"{key}: {value}")


In [ ]:
import pyomo.environ as pe
from pyomo.opt import SolverFactory
from pyomo.environ import DataPortal
import itertools
import time
def solve_tsp(Cities,Mode):
    
        # Ensure Cities is defined and passed correctly
    if not Cities:
        raise ValueError("Cities list must be provided")
    
    model = pe.AbstractModel()
    model.cities = pe.Set(initialize=Cities)
    model.sub_cities = pe.Set(initialize=Cities[1:])
    model.CITIES = pe.Set()
    model.transport_mode = pe.Set(initialize=[1,2,3,4])
    model.DISTANCE = pe.Param(model.CITIES, model.CITIES, within=pe.Reals)  
    model.L = pe.Param(model.transport_mode, within=pe.Reals, initialize={1:(1000), 2:(2000), 3:(6000), 4:(6000)})
    
    data = DataPortal()
    data.load(filename='C:/Users/aayush/Downloads/Polyhouse/DistanceBig.dat', model=model)
    
    data_dict = {}
    for i in Cities:
        for j in Cities:
            data_dict[(i,j)] = data['DISTANCE'][i,j]
    model.d_ij = pe.Param(model.cities, model.cities, within=pe.Reals, initialize=data_dict)
    
    # Initialize Z_dict
    Z_dict = {}

    for i in Cities:
        for j in Cities:
            if i == Cities[0] and j == Cities[0]:
                Z_dict[j] = 0
            elif i == Cities[0] and j != Cities[0]:
            # Sum over the variable values for x_ikjp
                sum_x = sum(ampl.getVariable('x_ikjp')[i, Mode, j, p].value() for p in ampl.getSet('P'))
                if sum_x < 1:
                    Z_dict[j] = 0
                elif sum_x > 1:
                    Z_dict[j] = sum_x
    
    model.Z =  pe.Param(model.cities, within=pe.Reals, initialize=Z_dict)
 
    model.y = pe.Var(model.cities, model.cities, within=pe.Binary)
    model.w = pe.Var(model.cities, model.cities, within=pe.NonNegativeIntegers)
    model.u = pe.Var(model.cities, within=pe.NonNegativeReals)
    
    
    def obj_expression(model):
        return sum(model.d_ij[i,j]*model.w[i,j] for i in model.cities for j in model.sub_cities) + model.L[Mode]*sum(model.d_ij[i,j]*model.y[i,j] for i in model.cities for j in model.sub_cities) + model.L[Mode]*sum(model.d_ij[i,Cities[0]]*model.y[i,Cities[0]] for i in model.sub_cities)
    model.obj = pe.Objective(expr=obj_expression, sense=pe.minimize)

    
    def con1_rule(model, i):
        return sum(model.y[i,j] for j in model.cities if j != i) == 1
    model.con1 = pe.Constraint(model.cities, rule=con1_rule)

    def con2_rule(model, j):
        return sum(model.y[i,j] for i in model.cities if i != j) == 1
    model.con2 = pe.Constraint(model.cities, rule=con2_rule)

    def no_self_loops(model, i):
        return model.y[i,i] == 0
    model.no_self_loops = pe.Constraint(model.cities, rule=no_self_loops)
    
    
    # Define the MTZ subtour elimination constraint
    def mtz_subtour_elimination_rule(model, i, j):
        if i != j and i!=Cities[0]:
            return model.u[j] - model.u[i] + model.y[i,j]*len(Cities) <= len(Cities) - 1 
        else:
            return pe.Constraint.Skip
    model.mtz_subtour_elimination = pe.Constraint(model.cities, model.cities, rule=mtz_subtour_elimination_rule)
    
    
    def con6_rule(model,i,j):
        if i != j:
            return model.w[i,j] >= sum(model.w[k,i] for k in model.cities if k != i and k!= j) - model.Z[i] - sum(model.Z[k] for k in model.cities)*(1-model.y[i,j])
        else:
            return pe.Constraint.Skip
    model.con6 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con6_rule)

    def con7_rule(model,j):
        return model.w[Cities[0],j] == model.y[Cities[0],j]*sum(model.Z[i] for i in model.cities)
    model.con7 = pe.Constraint(model.sub_cities, rule=con7_rule)
    
    def con8_rule(model,i,j):
        if i != j:
            return model.w[i,j] >= model.Z[j]*model.y[i,j]
        else:
            return pe.Constraint.Skip
    model.con8 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con8_rule)

    def con9_rule(model,i,j):
        if i != j:
            return model.w[i,j] <= model.y[i,j]*sum(model.Z[k] for k in model.cities)
        else:
            return pe.Constraint.Skip
    model.con9 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con9_rule)

    
    instance2 = model.create_instance()
    solver = SolverFactory('gurobi')
    
    start_time1 = time.time()
    
    results = solver.solve(instance2)
    end_time1 = time.time()

    for i in instance2.model().cities:
        for j in instance2.model().cities:
            if instance2.y[i,j].value > 0:
                print(f"y[{i},{j}]: {instance2.y[i,j].value}")
    

In [ ]:
# Running second phase model for each z*ik to find the optimal routing of transport mode k - 
# transporting products from particular polyhouse region i
for key in j_dict:
    print(key, ':', j_dict[key])
    print('Total Cities in the route:', len(j_dict[key]))
    if len(j_dict[key]) > 2:
        solve_tsp(j_dict[key],key[1])
    print('')

In [ ]:
import pandas as pd
# Create a DataFrame to store the results
df = pd.DataFrame(columns=['Polyhouse', 'Transport Mode', 'Distribution Center', 'Product', 'x'])

# Extract the variable values from AMPL
x_var = ampl.getVariable('x_ikjp')

# Iterate over the variable values and add them to the DataFrame
for idx in x_var.getValues().toList():
    i, k, j, p = idx[0], idx[1], idx[2], idx[3]
    x_value = idx[4]
    if x_value > 0:
        df1 = pd.DataFrame({'Polyhouse': [i], 'Transport Mode': [k], 'Distribution Center': [j], 'Product': [p], 'x': [x_value]})
        df = pd.concat([df, df1], ignore_index=True)

# Print the DataFrame
print(df)

# Display all rows of the DataFrame
pd.set_option('display.max_rows', None)

In [ ]:
df2 = df[df['x'] > 0].groupby('Polyhouse')['x'].sum().to_frame().T
df3 = df[df['x'] > 0].groupby('Distribution Center')['x'].sum().to_frame().T
df4 = df[df['x'] > 0].groupby('Product')['x'].sum().to_frame().T
df5 = df[df['x'] > 0].groupby('Transport Mode')['x'].sum().to_frame().T

df.to_excel("C:/Users/aayush/Desktop/Polyhouse/ResultM.xlsx")
df=pd.read_excel("C:/Users/aayush/Desktop/Polyhouse/ResultM.xlsx")